In [2]:
from random import *
import numpy as np

In [3]:
def func(x,y): #la fonction qu'on va chercher à minimiser, stmt convexe donc facile à minimiser
    return(x**2 + y**2)




def recuit_simule(start,f=func,nb_steps=100000,T=0.1):
    (x,y)=start
    for i in range(nb_steps):
        delta_x = 2*randint(0,1)-1 #saut de +1 ou -1 en x
        delta_y = 2*randint(0,1)-1 #saut de +1 ou -1 en y
        diff = f(x+delta_x, y+ delta_y) - f(x,y)
        if diff > 0:
            a = random()
            if a < np.exp(-diff/T):
                (x,y) = (x+delta_x, y+ delta_y)
        else:
            (x,y) = (x+delta_x, y+ delta_y)
    
    return(x,y)

# Test du recuit sans parallelisation

In [4]:
import time

t0 = time.time()

recuit_simule((-10,10))

t1 = time.time()
print(t1-t0)

0.4305541515350342


# Parallelisation avec multiprocessing

In [5]:
import multiprocessing as mp

#on lance plusieurs recuits en parallèle, initialisés à des endroits différents, dans l'idée de prendre ensuite le minimum des résultats

#une autre parallelisation que l'on pourrait tester est de lancer plusieurs évaluations (en parrallele) de la fonction f à chaque étape
#(par exemple si l'évaluation de f est coûteuse), et on ne retient que la valeur minimale

starts= [(randint(-100,100), randint(-100,100)) for i in range(8)] #different points de départ

pool = mp.Pool(mp.cpu_count())

t0 = time.time()

results = [pool.apply(recuit_simule, args=(start)) for start in zip(starts)]
pool.close()    

t1 = time.time()
print(t1-t0)

3.1343581676483154


# Parallelisation avec threading

In [6]:
import threading
import time


class MonThread (threading.Thread):
    def __init__(self, start_point, num_thread, f=func,nb_steps=1000000,T=0.1):
        threading.Thread.__init__(self)
        self.start_point=start_point
        self.f = f
        self.nb_steps=nb_steps
        self.T = T
        self.num = num_thread

    def run(self):
        t0 = time.time()
        (x,y)=self.start_point
        for i in range(self.nb_steps):
            delta_x = 2*randint(0,1)-1
            delta_y = 2*randint(0,1)-1
            diff = self.f(x+delta_x, y+ delta_y) - self.f(x,y)
            if diff > 0:
                a = random()
                if a < np.exp(-diff/self.T):
                    (x,y) = (x+delta_x, y+ delta_y)
            else:
                (x,y) = (x+delta_x, y+ delta_y)
        t1 = time.time()
        print("Thread numéro " +str(int(self.num)) + " terminé en " + str(t1-t0) + "s, résultat: (" + str(x) + "," + str(y)+") \n")
        return(x,y)

nb_threads = 4
    
starts= [(randint(-100,100), randint(-100,100)) for i in range(nb_threads)]

threads = [MonThread(starts[i],i) for i in range(nb_threads)]
threads[0].start()
threads[1].start()
threads[2].start()
threads[3].start()

Thread numéro 2 terminé en 15.248472929000854s, résultat: (0,-1) 

Thread numéro 3 terminé en 15.272424936294556s, résultat: (-1,0) 

Thread numéro 1 terminé en 15.384188175201416s, résultat: (0,1) 

Thread numéro 0 terminé en 15.85901403427124s, résultat: (0,0) 

